In [6]:
import os
import mlflow
import warnings
import pandas as pd
import numpy as np
import psycopg2 as pg
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
warnings.filterwarnings("ignore")

In [13]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: solid;
    }
</style>

In [7]:
params_dict = {
    'host': os.getenv('HOST_CDS'),
    'port': os.getenv('PORT_CDS'),
    'database': os.getenv('DATABASE_CDS'),
    'username': os.getenv('USERNAME_CDS'),
    'pwd': os.getenv('PWD_CDS')
}

In [8]:
host = 'localhost'    #params_dict['host']
port =  5432              #params_dict['port']
database = 'sample_db'   #params_dict['database']
username =  'postgres'            #params_dict['username']
pwd =  'postgress'                  # params_dict['pwd']

In [10]:
conn = pg.connect(user=username, password=pwd, host=host, port=port, database=database)

In [11]:
query_table_product = '''
    select * from product;
'''

In [12]:
df_product = pd.read_sql(query_table_product, conn)
df_product.head()

,product_id,product_name,product_class,category_hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0


In [14]:
df_product

,product_id,product_name,product_class,category_hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0
...,...,...,...,...,...,...,...,...,...
42989,42989,malibu pressure balanced diverter fixed shower...,Shower Panels,Home Improvement / Bathroom Remodel & Bathroom...,the malibu pressure balanced diverter fixed sh...,producttype : shower panel|spraypattern : rain...,3.0,4.5,2.0
42990,42990,emmeline 5 piece breakfast dining set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,NaN,basematerialdetails : steel| : gray wood|ofhar...,1314.0,4.5,864.0
42991,42991,maloney 3 piece pub table set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,this pub table set includes 1 counter height t...,additionaltoolsrequirednotincluded : power dri...,49.0,4.0,41.0
42992,42992,fletcher 27.5 '' wide polyester armchair,Teen Lounge Furniture|Accent Chairs,Furniture / Living Room Furniture / Chairs & S...,"bring iconic , modern style to your space in a...",legmaterialdetails : rubberwood|backheight-sea...,1746.0,4.5,1226.0


In [17]:
query_table_query = '''
    select * from queries;
'''

In [22]:
df_query = pd.read_sql(query_table_query, conn)
df_query.head()

,query_id,query,query_class
0,0,salon chair,Massage Chairs
1,1,smart coffee table,Coffee & Cocktail Tables
2,2,dinosaur,Kids Wall Décor
3,3,turquoise pillows,Accent Pillows
4,4,chair and a half recliner,Recliners


In [20]:
query_table_labels = '''
    select * from labels;
'''

In [21]:
df_labels = pd.read_sql(query_table_labels, conn)
df_labels

,id,query_id,product_id,label,label_score
0,0,0,25434,Exact,1.00
1,1,0,12088,Irrelevant,0.00
2,2,0,42931,Exact,1.00
3,3,0,2636,Exact,1.00
4,4,0,42923,Exact,1.00
...,...,...,...,...,...
233443,234010,478,15439,Partial,0.75
233444,234011,478,451,Partial,0.75
233445,234012,478,30764,Irrelevant,0.00
233446,234013,478,16796,Partial,0.75


In [26]:
query_table_product = '''
SELECT 
    product_id,
    product_name,
    COALESCE(product_description, product_name) AS product_text
FROM 
    product;
'''

# Read the query result into a pandas DataFrame
product_text_pd  = pd.read_sql(query_table_product, conn)

# Display the first few rows of the DataFrame
print(df_product_text.head())

   product_id                                       product_name  \
0           0                            solid wood platform bed   
1           1                        all-clad 7 qt . slow cooker   
2           2            all-clad electrics 6.5 qt . slow cooker   
3           3       all-clad all professional tools pizza cutter   
4           4  baldwin prestige alcott passage knob with roun...   

                                        product_text  
0  good , deep sleep can be quite difficult to ha...  
1  create delicious slow-cooked meals , from tend...  
2  prepare home-cooked meals on any schedule with...  
3  this original stainless tool was designed to c...  
4  the hardware has a rich heritage of delivering...  


In [27]:
product_text_pd

,product_id,product_name,product_text
0,0,solid wood platform bed,"good , deep sleep can be quite difficult to ha..."
1,1,all-clad 7 qt . slow cooker,"create delicious slow-cooked meals , from tend..."
2,2,all-clad electrics 6.5 qt . slow cooker,prepare home-cooked meals on any schedule with...
3,3,all-clad all professional tools pizza cutter,this original stainless tool was designed to c...
4,4,baldwin prestige alcott passage knob with roun...,the hardware has a rich heritage of delivering...
...,...,...,...
42989,42989,malibu pressure balanced diverter fixed shower...,the malibu pressure balanced diverter fixed sh...
42990,42990,emmeline 5 piece breakfast dining set,NaN
42991,42991,maloney 3 piece pub table set,this pub table set includes 1 counter height t...
42992,42992,fletcher 27.5 '' wide polyester armchair,"bring iconic , modern style to your space in a..."


In [28]:
original_model = SentenceTransformer('all-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L12-v2')

In [31]:
documents = (
  DataFrameLoader(
    product_text_pd,
    page_content_column='product_text'
    )
    .load()
  )

In [32]:
vectordb = Chroma.from_documents(
  documents=documents, 
  embedding=embedding_model
  )
 
# persist vector db to storage
vectordb.persist()

In [33]:
vectordb._collection.count()

42994

In [34]:
rec= vectordb._collection.peek(1)
 
print('Metadatas:  ', rec['metadatas'])
print('Documents:  ', rec['documents'])
print('ids:        ', rec['ids'])
print('embeddings: ', rec['embeddings'])

Metadatas:   [{'product_id': 9499, 'product_name': 'real touch magnolia , berry & cypress mixed floral arrangement in glass vase'}]
Documents:   ['we are taking forever flowers to the next level with our new line of real touch flower arrangements in luxury glassware . our red real touch magnolia , berry & cypress christmas arrangement in glass cylinder vase is the perfect statement piece for your kitchen , dining room , office , or any space that needs a little life ! these arrangements also makes for an exquisite gift that can be enjoyed for years to come . our artificial water is precisely formulated to simulate the look of clean , fresh water . it remains clear after hardening to hold each arrangement in place .']
ids:         ['000217d9-ccd3-4dc6-8fb3-0d9d76c4646c']
embeddings:  [[-0.051275234669446945, -0.00912544596940279, 0.07982894778251648, 0.0279383547604084, 0.013285013847053051, -0.06080647185444832, -0.0215378999710083, -0.03740846365690231, -0.01724035106599331, 0.0176404

In [35]:
vectordb.similarity_search_with_score("kid-proof rug")

[(Document(page_content='this modern and convenient rug combines contemporary colors , an elegant medallion pattern , a wood floor safe backing , and durable construction . the kid and pet safe materials are easy to spot clean , making these rugs ideal for a busy family .', metadata={'product_id': 41813, 'product_name': 'nile tarifa bohemian medallion gray area rug'}),
  0.5344257354736328),
 (Document(page_content='this modern and convenient rug combines contemporary colors , an elegant medallion pattern , a wood floor safe backing , and durable construction . the kid and pet safe materials are easy to spot clean , making these rugs ideal for a busy family .', metadata={'product_id': 41814, 'product_name': 'nile tarifa bohemian medallion rust area rug'}),
  0.5344257354736328),
 (Document(page_content='this modern and convenient rug combines contemporary colors , an elegant medallion pattern , a wood floor safe backing , and durable construction . the kid and pet safe materials are ea

In [41]:
from sentence_transformers import SentenceTransformer, util, InputExample, losses, evaluation
import torch
from torch.utils.data import DataLoader
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import numpy as np
import pandas as pd
import mlflow

In [43]:
query = '''
SELECT 
    q.query,
    COALESCE(p.product_description, p.product_name) AS product_text,
    l.label_score AS score
FROM 
    product p
JOIN 
    labels l ON p.product_id = l.product_id
JOIN 
    queries q ON l.query_id = q.query_id;
'''

# Read the query result into a pandas DataFrame
search_pd = pd.read_sql(query, conn)

# Display the first few rows of the DataFrame
print(search_pd.head())

         query                                       product_text  score
0  salon chair  add a beautiful accent to any room with this m...    0.0
1  salon chair  the heavy duty barber chair is built to last ....    1.0
2  salon chair  welcome your corporate visitors with this thre...    0.0
3  salon chair  stylish fits perfectly for office , bank , hal...    0.0
4  salon chair                                                NaN    0.0


In [44]:
search_pd.head()

,query,product_text,score
0,salon chair,add a beautiful accent to any room with this m...,0.0
1,salon chair,the heavy duty barber chair is built to last ....,1.0
2,salon chair,welcome your corporate visitors with this thre...,0.0
3,salon chair,"stylish fits perfectly for office , bank , hal...",0.0
4,salon chair,NaN,0.0


In [45]:
original_model = SentenceTransformer('all-MiniLM-L12-v2')

In [46]:
query_embeddings = (
  original_model
    .encode(
      search_pd['query'].tolist()
      )
  )

In [47]:
product_embeddings = (
  original_model
    .encode(
      search_pd['product_text'].tolist()
      )
  )

In [50]:
# determine cosine similarity for each query-product pair
original_cos_sim_scores = (
  util.pairwise_cos_sim(
    query_embeddings, 
    product_embeddings
    )
  )

In [51]:
# average the cosine similarity scores
original_cos_sim_score = torch.mean(original_cos_sim_scores).item()
 
# display result
print(original_cos_sim_score)

0.37226954102516174


In [52]:
# determine correlation between cosine similarities and relevancy scores
original_corr_coef_score = (
  np.corrcoef(
    original_cos_sim_scores,
    search_pd['score'].values
  )[0][1]
) 
# print results
print(original_corr_coef_score)

0.3192255803956717


Step 2: Fine-Tune the Model

Restructure Data for Model Input

In [53]:
# define function to assemble an input
def create_input(doc1, doc2, score):
  return InputExample(texts=[doc1, doc2], label=score)
 
# convert each search result into an input
inputs = search_pd.apply(
  lambda s: create_input(s['query'], s['product_text'], s['score']), axis=1
  ).to_list()

In [54]:
tuned_model = SentenceTransformer('all-MiniLM-L12-v2')

Tune the Model

In [55]:
# define instructions for feeding inputs to model
input_dataloader = DataLoader(inputs, shuffle=True, batch_size=16) # feed 16 records at a time to the model
 
# define loss metric to optimize for
loss = losses.CosineSimilarityLoss(tuned_model)
 
# tune the model on the input data
tuned_model.fit(
  train_objectives=[(input_dataloader, loss)],
  epochs=1, # just make 1 pass over data
  warmup_steps=100 # controls how many steps over which learning rate increases to max before descending back to zero
  )

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/14591 [00:00<?, ?it/s]

KeyboardInterrupt: 